# Best practices in Flask

## Handling URL parameter types

When you specify a URL route in Flask you have a few options for how to handle the *type* of data handled by that path.

Let say that we had an endpoint that must only accept an integer in the path, for example:

`/invoice_number/<num>/`

If you made an endpoint that looked like this:

In [ ]:

invoices = {
    1: "invoice 1",
    2: "invoice 2",
}

@app.route("/invoice_number/<num>/")
def get_invoice(num):
    """Returns the details for the given invoice number"""
    invoice_info = invoices[num]
    return f"Invoice number {num}: {invoice_info}"

There's a few subtle bugs that are present here.
If we didn't do some processing first we would have a `KeyError` any time that someone didn't provide a number.

### URL Variable rules

A good way to handle this is to use Flask's path parameter type support to automatically 404 on any URL path items that are the wrong type.
This give us the benefits of not having to write the type checking code ourselves and also preventing bugs from happening in our code because the function is never run when the path is the wrong type.

format | Accepts
-      | -
string | (default) accepts any text without a slash
int    | accepts positive integers
float  | accepts positive floating point values
path   | like string but also accepts slashes
uuid   | accepts UUID strings

We can use this to more narrowly specify what path variable types are acceptable like so:

In [ ]:

invoices = {
    1: "invoice 1",
    2: "invoice 2",
}

@app.route("/invoice_number/<num:int>/")
def get_invoice(num: int):
    """Returns the details for the given invoice number"""
    invoice_info = invoices[num]
    return f"Invoice number {num}: {invoice_info}"

Now if we attempt to access this endpoint with `/invoice_number/NotNumber` we will get a 404 error immediately.

## Logging in Flask

Flask uses the built in Python logging library to handle logging. This is made accessible via `app.logger`.

The documentation states the following about setting up logging:

> When you want to configure logging for your project, you should do it as soon as possible when the program starts. If app.logger is accessed before logging is configured, it will add a default handler. If possible, configure logging before creating the application object.

Since Flask is using the default Python logging library you can use those same general techniques you would have used outside Flask.
Please refer to the general logging materials for more details.

One common pattern is to register a `logging.handlers.SMTPHandler` to automatically send an email to the website administration team when sufficiently critical errors are encountered. The pattern will look like this:

In [ ]:
import logging
from logging.handlers import SMTPHandler

mail_handler = SMTPHandler(
    mailhost='127.0.0.1',
    fromaddr='[email protected]',
    toaddrs=['[email protected]'],
    subject='Application Error'
)
mail_handler.setLevel(logging.ERROR)
mail_handler.setFormatter(logging.Formatter(
    '[%(asctime)s] %(levelname)s in %(module)s: %(message)s'
))

if not app.debug:
    app.logger.addHandler(mail_handler)

Note that for this to work you must have the SMTP server set up on the same machine that is running the Flask app.

## Error handlers

One of the design decisions with Flask was the ability to use exceptions to represent HTTP error codes, this means that in many cases Exceptions are being directly used to control the flow of the web applications. Here's how we can use the `abort` function to represent a 404 error via the exception it will raise.

In [ ]:
from flask import Flask, abort
app = Flask(__name__)
invoices = {
    1: "invoice 1",
    2: "invoice 2",
}

@app.route("/invoice_number/<num:int>/")
def get_invoice(num: int):
    """Returns the details for the given invoice number"""
    try:
        invoice_info = invoices[num]
    except KeyError:
        abort(404) # immediately raise a 404 exception
    return f"Invoice number {num}: {invoice_info}"

If one of these Flask HTTP exceptions propagates Flask will attempt to handle this by calling the error handler for that particular status code.

If you have an unhandled Python exception in your code the Flask framework will catch this exception and give a generic 500 error message to the user.
If you are writing your own code it makes sense to hook into this system to do important things like logging.

### Dealing with unhandled Python exceptions

Whenever we have an unhandled Python exception in our application we want to make sure that we know about it via logging and other means.

We can use `@app.errorhandler` to catch Python exceptions but since flask uses exceptions to indicate non-success HTTP codes we have to be careful with what we catch:

Since the Flask HTTP exceptions derive from `werkzeug.exceptions.HTTPException` we have to make sure we do *not* catch those:

In [ ]:
from werkzeug.exceptions import HTTPException #base for all flask HTTP exceptions

@app.errorhandler(Exception)
def handle_exception(e):
    # pass through Flask Exceptions
    if isinstance(e, HTTPException):
        return e

    # All exceptions here are uncaught non-Flask exceptions
    # Log at some high severity here
    # Also perhaps roll back any DB transactions in progress here
    return render_template('500_generic.html', e=e), 500

Note that if we were serving an API we would generate something like a JSON response here.

## Reversing URLs

When we are writing functions that will handle our URL endpoints we sometimes want to be able to find the URL that is being handled for a given function.

For example, let's say we have an endpoint like this:

In [ ]:
from flask import Flask
app = Flask(__name__)
@app.route("/login")
def authenticate():
    return "This is the login page!"

@app.route("/")
def home():
    auth_url = "/login"
    return f"The auth page can be found at: {auth_url}:

This particular way of generating the contents of `auth_url` is problematic because in the future that URL route could change.
To deal with this in a reliable way Flask provides the `url_for` helper to find out what URL route is associated with a python function:

In [ ]:
from flask import Flask, url_for

@app.route("/login")
def authenticate():
    return "This is the login page!"

@app.route("/")
def home():
    auth_url = url_for(authenticate)
    return f"The auth page can be found at: {auth_url}:

Now the URL will always be properly generated, if we changed `/login` to `/authenticate` then the `/` route will always display the right URL to find our `authenticate` function.

### Exercise: dynamically create a sitemap for the following

In [ ]:
users = ["Ed", "Henry", "Errol", "Janis"]
products = [1,2,3,4]

@app.route("/products")
def products():
    """Return products information
    sitemap: product information
    """
    return products

@app.route("/users")
def users():
    """Return user information
    sitemap: user information
    """
    return users

@app.route("/login")
def authenticate():
    """
    Authenticate a user
    sitemap: This is the login page
    """
    return "This is the login page!"

@app.route("/")
def home():
    """Homepage
    sitemap: homepage
    """
    auth_url = url_for(authenticate)
    return f"The auth page can be found at: {auth_url}:

- Hint: The function documentation can be accessed via `__doc__`
- Hint: you can access the names defined in the global state via `globals()`